# Camera to Machine Calibration
This notebook steps you through the calibration the pixel-space of the camera with the real-world machine coordinates, so that Jubilee can accurately move to a point in an image. Adapted with credit to Matthew Sorensen https://github.com/matthewsorensen/jubileeautofocus

### 0. Notebook Setup

In [1]:
# import required modules
import sys
sys.path.append('..')
import os
import numpy as np
import time
import math
import json
import ipywidgets as widgets
import ipympl


import cv2
from utils.MachineUtils import *
from utils.CameraUtils import * 

In [2]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [3]:
# Choose the correct port from above and establish connection with machine
port = '/dev/ttyACM0'
m = MachineCommunication(port)

In [ ]:
# Use the config file for our current machine hardware
def search_up_dirs(target_dir, max_cycles):
    n = 0
    while n != max_cycles:
        curr_folder = os.path.basename(os.path.normpath(os.getcwd()))
        if curr_folder == target_dir:
            break
        os.chdir('..')
        n = n + 1
        
search_up_dirs('duckbot', 5) #Shift to duckbot
os.chdir('ConfigFiles')
labware_config_file_dir = os.path.join(os.getcwd(),'HardwareLabwareConfigs')
os.chdir(labware_config_file_dir)
config_opts = os.listdir(os.getcwd())
hl_choice = widgets.Dropdown(options = config_opts)
display(hl_choice)

In [ ]:
with open(hl_choice.value,'r') as datafile:
    hardwarelabwareconfig = json.load(datafile)
    
tool_positions = hardwarelabwareconfig['tool_positions']
print(tool_positions)

### 1. Home your machine
Make sure the build plate is clear!

In [ ]:
m.homeAll()

### 2. Set your height for calibration
This procedure calibrates *for a particular z height*. You should choose the best height for the images you want to take on your machine. 

In [ ]:
m.toolChange(tool_positions['camera'])

In [ ]:
# change this height depending on your application
# for this example, we will be calibrating at height of z=10, for close imaging of individual well plates
m.moveTo(z=10)

### 3. Print a calibration circle
We will use a printed circle to conduct the calibration. Move the camera over an open spot on your bed and focus the camera. Then, determine a suitable size circle to print out which fits within the resulting picture. 
ToDo: link to illustrator/inkscape file to make the circle, or make it directly here in jupyter

In [ ]:
# move to an open spot on the bed
m.moveTo(x=97, y=150)

In [64]:
# focus the camera
# choose the correct video device if you have >1 camera
# center the tool under the microscope
# make note of the x,y coordinates
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

(320, 240)


In [ ]:
# It's important that your calibration circle doesn't move during the calibration process
# Print a circle at a size suitable for your z height, and tape it to the bed plate

## 4. Find the size of the printed circle in pixels
To accurately detect the circle, we'll find the radius of the circle in pixels

In [ ]:
%matplotlib tk #trying different backends


# take a picture
f = getFrame()

# Try finding a single circle in the image
# If the circle is much bigger or smaller than the bounds here, change them until a single circle is found
circle, circleData = getSingleWell(f, minR = 0, maxR = 500)
showFrame(circle)

In [ ]:
# set the min and max radius bounds based on the results above
tolerance = 5
circleRadius = circleData[0][2] # circle data returns a list of (center_x, center_y, radius) coordinates
minRadius = circleRadius - tolerance
maxRadius = circleRadius + tolerance

### 5. Run the calibration
Using the parameters set above, we can now run the calibration procedure!

In [ ]:
# ToDo: port the calibration code

### 6. Check Calibration
We can move to a few points to confirm that the calibration looks good!
ToDo: file organization, where the calibration file gets written to

In [ ]:
# loads in the relevant calibration files

with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_10.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

m.transform = matrix
m.img_size = size

# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

In [ ]:
# move to the center of the calibration circle
center = m.transform.T @ np.array([0, 0, 0, 0, 0, 1])
m.moveTo(x=center[0], y=center[1])

In [ ]:
# check position
showFrame(getFrame())

In [ ]:
# ToDo: can't switch backends mid notebook for inline/gui  
f = getFrame()
pts = selectPoint(f, num_pts=3)

In [ ]:
for pt in pts:
    off = px_to_real(pt[0], pt[1])
    m.moveTo(x=center[0]-off[0], y=center[1]-off[1])
    showFrame(getFrame(), grid=True)
    input()

In [ ]:
### Testing Syringe Alignment
m.moveTo(z=50)
m.toolChange(3)

In [ ]:
m.moveTo(x=center[0], y=center[1])

In [ ]:
syringe_off = [-0.3, 1.1]

In [ ]:
m.moveTo(x=center[0] + syringe_off[0], y=center[1]+syringe_off[1])

In [65]:
# fnum = 1

In [114]:
f = getFrame()

In [115]:
print(fnum)
showFrame(f)
print(fnum)

15
15


In [116]:
saveFrame(f, f"/home/pi/Downloads/pscope-hex-circles-{fnum}.png")
fnum += 1